In [1]:
import pandas as pd
import json
from pathlib import Path

path = Path('..', 'logs', 'app.log')

# read the logs file into memory
text = path.read_text()

# parse the json
data = [json.loads(l) for l in text.splitlines()]

# create a dataframe
df = pd.DataFrame(data)

# Set pandas to display all columns for one row
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


df

,name,msg,args,levelname,levelno,pathname,filename,module,exc_info,exc_text,stack_info,lineno,funcName,created,msecs,relativeCreated,thread,threadName,processName,process,message,asctime
0,root,CLEARED LOG FILE AS production ENVIRONMENT VARIABLE IS NOT SET,[],WARNING,30,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,24,<module>,1.698614e+09,662.0,2328.657150,2636,MainThread,MainProcess,24296,CLEARED LOG FILE AS production ENVIRONMENT VARIABLE IS NOT SET,"2023-10-29 14:06:50,662"
1,root,SA_PASSWORD environment variable is set,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,35,<module>,1.698614e+09,684.0,2350.658894,2636,MainThread,MainProcess,24296,SA_PASSWORD environment variable is set,"2023-10-29 14:06:50,684"
2,root,creating flask app,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,37,<module>,1.698614e+09,686.0,2352.659941,2636,MainThread,MainProcess,24296,creating flask app,"2023-10-29 14:06:50,686"
3,root,creating filemanager,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,49,<module>,1.698614e+09,695.0,2361.680031,2636,MainThread,MainProcess,24296,creating filemanager,"2023-10-29 14:06:50,695"
4,camera.sqlmanager,"SQLManager does not exist, creating new instance",[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\camera\sqlmanager.py,sqlmanager.py,sqlmanager,None,None,None,114,__new__,1.698614e+09,696.0,2362.657309,2636,MainThread,MainProcess,24296,"SQLManager does not exist, creating new instance","2023-10-29 14:06:50,696"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,camera.filemanager,wrote frame 298 to data\webcam\video_cache\20231029_140653_Pacific Daylight Time..mp4 with guid 6673b729-c79d-4ef1-b49d-10348f796a37 at 1698613643.9025843,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\camera\filemanager.py,filemanager.py,filemanager,None,None,None,82,write_line,1.698614e+09,902.0,35568.802118,20040,Filemanager_Thread,MainProcess,24296,wrote frame 298 to data\webcam\video_cache\20231029_140653_Pacific Daylight Time..mp4 with guid 6673b729-c79d-4ef1-b49d-10348f796a37 at 1698613643.9025843,"2023-10-29 14:07:23,902"
378,camera.filemanager,wrote frame 299 to data\webcam\video_cache\20231029_140653_Pacific Daylight Time..mp4 with guid 387633f6-c4ff-4434-aa96-2f775f5fdb93 at 1698613644.003575,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\camera\filemanager.py,filemanager.py,filemanager,None,None,None,82,write_line,1.698614e+09,3.0,35669.792891,20040,Filemanager_Thread,MainProcess,24296,wrote frame 299 to data\webcam\video_cache\20231029_140653_Pacific Daylight Time..mp4 with guid 387633f6-c4ff-4434-aa96-2f775f5fdb93 at 1698613644.003575,"2023-10-29 14:07:24,003"
379,werkzeug,[31m[1mWARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.[0m\n * Running on http://127.0.0.1:5000,[],INFO,20,d:\Documents\.vscode\SecurityCamera\env\Lib\site-packages\werkzeug\_internal.py,_internal.py,_internal,None,None,None,224,_log,1.698614e+09,752.0,45419.049740,2636,MainThread,MainProcess,24296,[31m[1mWARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.[0m\n * Running on http://127.0.0.1:5000,"2023-10-29 14:07:33,752"
380,werkzeug,[33mPress CTRL+C to quit[0m,[],INFO,20,d:\Documents\.vscode\SecurityCamera\env\Lib\site-packages\werkzeug\_internal.py,_internal.py,_internal,None,None,None,224,_log,1.698614e+09,756.0,45423.041821,2636,MainThread,MainProcess,24296,[33mPress CTRL+C to quit[0m,"2023-10-29 14:07:33,756"


In [2]:
# show the df head with all columns
df.head()

,name,msg,args,levelname,levelno,pathname,filename,module,exc_info,exc_text,stack_info,lineno,funcName,created,msecs,relativeCreated,thread,threadName,processName,process,message,asctime
0,root,CLEARED LOG FILE AS production ENVIRONMENT VARIABLE IS NOT SET,[],WARNING,30,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,24,<module>,1.698614e+09,662.0,2328.657150,2636,MainThread,MainProcess,24296,CLEARED LOG FILE AS production ENVIRONMENT VARIABLE IS NOT SET,"2023-10-29 14:06:50,662"
1,root,SA_PASSWORD environment variable is set,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,35,<module>,1.698614e+09,684.0,2350.658894,2636,MainThread,MainProcess,24296,SA_PASSWORD environment variable is set,"2023-10-29 14:06:50,684"
2,root,creating flask app,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,37,<module>,1.698614e+09,686.0,2352.659941,2636,MainThread,MainProcess,24296,creating flask app,"2023-10-29 14:06:50,686"
3,root,creating filemanager,[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\api\app.py,app.py,app,None,None,None,49,<module>,1.698614e+09,695.0,2361.680031,2636,MainThread,MainProcess,24296,creating filemanager,"2023-10-29 14:06:50,695"
4,camera.sqlmanager,"SQLManager does not exist, creating new instance",[],DEBUG,10,D:\Documents\.vscode\SecurityCamera\camera\sqlmanager.py,sqlmanager.py,sqlmanager,None,None,None,114,__new__,1.698614e+09,696.0,2362.657309,2636,MainThread,MainProcess,24296,"SQLManager does not exist, creating new instance","2023-10-29 14:06:50,696"


In [3]:
df_frame_rate = df[(df['lineno'] == 82)]

# Set pandas to display the full raw value of floats
pd.set_option('display.float_format', '{:.10f}'.format)
# round and set to int
df_frame_rate['created_rounded'] = df_frame_rate['created'].round().astype(int)

# groupdby created_rounded and get count of rows
series_framerate = df_frame_rate.groupby('created_rounded').count()['created']

# mean, median, min, max, mode, std, var of series_framerate
print(series_framerate.describe())

print('-'*80)

print(series_framerate)
# df_frame_rate[['created_rounded', 'created', 'message', 'relativeCreated']]

count   31.0000000000
mean     9.6774193548
std      0.9793568221
min      5.0000000000
25%     10.0000000000
50%     10.0000000000
75%     10.0000000000
max     10.0000000000
Name: created, dtype: float64
--------------------------------------------------------------------------------
created_rounded
1698613614     8
1698613615    10
1698613616    10
1698613617    10
1698613618     9
1698613619    10
1698613620    10
1698613621    10
1698613622    10
1698613623    10
1698613624    10
1698613625    10
1698613626     9
1698613627    10
1698613628    10
1698613629    10
1698613630    10
1698613631    10
1698613632    10
1698613633    10
1698613634     9
1698613635    10
1698613636    10
1698613637    10
1698613638    10
1698613639    10
1698613640    10
1698613641    10
1698613642    10
1698613643    10
1698613644     5
Name: created, dtype: int64


C:\Users\Celebrinborn\AppData\Local\Temp\ipykernel_6556\810834942.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frame_rate['created_rounded'] = df_frame_rate['created'].round().astype(int)


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
